<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unNormal_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 기본 세팅

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 코렙의 경우 설치 후 runtime  > restart 해야함

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

## Quest : 성별에 따라 구매한 과일의 가격 분포가 이분산을 보이는지?

### 가격분포의 정규분포 여부 계산

In [5]:
df_fruit = pd.read_csv('fruitstore_saleshistory.csv')
df_fruit[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207.0,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977.0,매장C,NaN,10:05


In [17]:
df_fruit = df_fruit.query('물품중분류 == "과일"') # 과일만!
df_fruit[['회원번호', '성별', '물품중분류','물품소분류', '구매금액']][:10]

,회원번호,성별,물품중분류,물품소분류,구매금액
0,272369856,여,과일,사과,22207.0
62,445131904,여,과일,건대추,22973.0
132,744508416,여,과일,귤(조생),11104.0
187,281649856,여,과일,귤(조생),11104.0
220,548171456,여,과일,배,16847.0
225,742641280,여,과일,귤(조생),11104.0
232,742641280,여,과일,사과,18953.0
306,553065728,여,과일,귤(조생),11104.0
331,1289439296,여,과일,산딸기,10912.0
391,680162752,여,과일,귤(조생),11104.0


In [20]:
# 결측치 확인
df_fruit[['회원번호', '성별', '물품중분류','물품소분류', '구매금액']].isnull().sum()
# 결측치 없음!

회원번호     0
성별       0
물품중분류    0
물품소분류    0
구매금액     0
dtype: int64

In [22]:
# 구매 금액(연속형) 정규성 검증 -> 5000개 이상 -> Anderson-normality Test
import scipy.stats as stats
stats.anderson(df_fruit['구매금액'])

AndersonResult(statistic=627.2244139555223, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=20992.940750297737, scale=17432.375563643738)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

검정 통계량(627.224) > critical values(0.787) 이므로 비정규 분포
-> 범주형 2집단 -> Wilcoxon Test(ranksums)

In [23]:
df_fruit_woman = df_fruit.query("성별 == '여' ")['구매금액']
df_fruit_man = df_fruit.query("성별 == '남' ")['구매금액']

In [24]:
stats.ranksums(df_fruit_woman,df_fruit_man )

RanksumsResult(statistic=1.6171801001501234, pvalue=0.10583941056456257)

### 결론
- pvalue > 0.05이므로 귀무가설 채택
- 성별에 따라 구매한 과일의 가격분포는 상관이 없다.